# Predict Median House price in a district in california based on given features

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from custom import converter,encode
# from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import os, sys
# CURRENT_DIR = os.path.dirname(os.path.abspath())
sys.path.append('.')

In [2]:
housing = pd.read_csv('/mnt/New_Volume/Work_From_Home/Data/housing.csv')
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [3]:
housing.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [4]:
x = housing.drop('median_house_value',axis = 1)
y = housing['median_house_value']

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2, random_state=42)

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

In [7]:
scaler = StandardScaler()
encoder = OneHotEncoder(sparse = False)

In [8]:
# class converter(BaseEstimator, TransformerMixin):
#     def fit(self,x,y=None):
#         return self
        
#     def transform(self, x,y=None):
#         data_num = np.array(x)[:,:8]
#         return data_num

In [9]:
# class encode (BaseEstimator,TransformerMixin):
#     def fit(self, x, y =None):
#         return self
#     def transform(self, x,y=None):
#         data_cat = np.array(x)[:,-1]
#         return data_cat.reshape(-1,1)

In [10]:
from sklearn.pipeline import Pipeline
pipe_num = Pipeline([('num_extract',converter()),('imputer', imputer),("scale",scaler)])
pipe_cat = Pipeline([('categorical_col',encode()),('one_hot', encoder)])

In [11]:
from sklearn.pipeline import FeatureUnion
union = FeatureUnion(transformer_list=[('numerical',pipe_num),("catrgorical",pipe_cat)])

# Splitting the data

In [12]:
data_prepared = union.fit_transform(x_train)
data_prepared

array([[ 1.27258718, -1.37281156,  0.34849024, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916027, -0.87669581,  1.61811817, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760475, -0.46014652, -1.95271027, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.5994702 , -0.75500798,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553913,  0.90651131, -1.07984114, ...,  0.        ,
         0.        ,  0.        ],
       [-1.4148972 ,  0.99543703,  1.8561734 , ...,  0.        ,
         1.        ,  0.        ]])

In [13]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()

In [14]:
# lin_reg.fit(data_prepared,y_train)

In [15]:
# y_pred = lin_reg.predict(union.transform(x_test)) 

In [16]:
final = Pipeline([('union',union),('linear_regression',lin_reg)])

In [17]:
final.fit(x_train,y_train)

Pipeline(steps=[('union',
                 FeatureUnion(transformer_list=[('numerical',
                                                 Pipeline(steps=[('num_extract',
                                                                  converter()),
                                                                 ('imputer',
                                                                  SimpleImputer(strategy='median')),
                                                                 ('scale',
                                                                  StandardScaler())])),
                                                ('catrgorical',
                                                 Pipeline(steps=[('categorical_col',
                                                                  encode()),
                                                                 ('one_hot',
                                                                  OneHotEncoder(sparse=False))]))])),
     

In [18]:
y_pred = final.predict(x_test)
y_pred

array([ 54576., 124992., 256064., ..., 438752., 121056., 182960.])

In [19]:
import pickle

pickle.dump(final,open("housing_linear.bin",'wb'))